In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import random
import sys
import json

In [2]:
sc = SparkContext()

In [3]:
sc.getConf().getAll()

[('spark.master',
  'spark://spark-60a43dd685736767aa903d9d-master-svc.domino-vip2-compute.svc.cluster.local:7077'),
 ('spark.driver.port', '33081'),
 ('spark.driver.host',
  'spark-driver-60a43dd685736767aa903d9d.domino-vip2-compute.svc.cluster.local'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.executor.memory', '4096m'),
 ('spark.ui.reverseProxyUrl',
  '/elliott_botwick/MLOps-Demo/workspaces/60a43dd685736767aa903d9d/spark/ui/'),
 ('spark.rdd.compress', 'True'),
 ('spark.executor.instances', '2'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.cores', '1'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'app-20210518231909-0001'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.ui.reverseProxy', 'true'),
 ('spark.ui.proxyBase', '.')]

In [4]:
sc

<SparkContext master=spark://spark-60a43dd685736767aa903d9d-master-svc.domino-vip2-compute.svc.cluster.local:7077 appName=pyspark-shell>

In [5]:
# from pyspark.conf import SparkConf
# SparkSession.builder.config(conf=SparkConf())

In [6]:
NUM_SAMPLES = 1000000
 
def inside(p):
 x, y = random.random(), random.random()
 return x*x + y*y < 1
count = sc.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()
pi = float(4.0 * count / float(NUM_SAMPLES))
print('Pi is roughly', pi)

Pi is roughly 3.139664


In [7]:
spark = SparkSession.builder.getOrCreate()

In [8]:
!ls /domino/datasets/local/testData

baseline.csv  churndata.csv  prepared.csv


In [9]:
df = spark.read.format('csv').options(infer_schema = True, header = True).load(('../..//domino/datasets/local/testData/prepared.csv'))


In [10]:
df.show(10)

+------+--------------------+----+-------------------+------------+----+------+------------------+------------------+---+------+-------+-----+---------+---------+--------+----+-------+
|custid|            dropperc|mins|      KPI_DeviceSat|consecmonths|long|income|   KPI_NetworkQual|          dataperc|age|  bill|datause|calls|education|servcalls|dataplan|kids|churn_Y|
+------+--------------------+----+-------------------+------------+----+------+------------------+------------------+---+------+-------+-----+---------+---------+--------+----+-------+
|844336|  0.0163636363636364| 550| 0.0160345039664219|          28|  35|  89.2|0.7603546409999999|    0.158935546875| 45| 71.44|    651|   55|       11|        0|    4096|   2|      0|
|146041|0.018348623853211003| 545| 0.0170551148054854|          33|  35|  54.2|       0.526147204|      0.3779296875| 43|124.23|    774|   66|        9|        0|    2048|   1|      0|
|847745|  0.0185185185185185| 378|0.00889616180366376|          41|  19|  5

In [11]:
df.count()

7939

In [23]:
s3_access_key = os.environ['AWS_ACCESS_KEY_ID']
s3_secret_key = os.environ['AWS_SECRET_ACCESS_KEY']
s3 = boto3.resource("s3", aws_access_key_id=s3_access_key, aws_secret_access_key=s3_secret_key)

In [19]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
sqlContext.read.csv('s3://seteam-files/mq/2016.csv')

Py4JJavaError: An error occurred while calling o242.csv.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:547)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:545)
	at scala.collection.TraversableLike$$anonfun$fl

In [ ]:
red_wine = spark.read.csv('test.csv')

In [12]:
#Setup Pyspark
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("MyAppName") \
        .getOrCreate()

# You can examine the full config
spark.sparkContext.getConf().getAll()

[('spark.ui.proxyBase', '.'),
 ('spark.driver.port', '39741'),
 ('spark.driver.host',
  'spark-driver-60245d4a0ac6130d45aab9ea.domino-vip2-compute.svc.cluster.local'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'app-20210210222631-0000'),
 ('spark.master',
  'spark://spark-60245d4a0ac6130d45aab9ea-master-svc.domino-vip2-compute.svc.cluster.local:7077'),
 ('spark.executor.memory', '4096m'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.instances', '1'),
 ('spark.app.name', 'MyAppName'),
 ('spark.executor.cores', '1'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.ui.reverseProxy', 'true'),
 ('spark.ui.reverseProxyUrl',
  '/elliott_botwick/SparkWineQuality/workspaces/60245d4a0ac6130d45aab9ea/spark/ui/')]

In [13]:
#Setup and Test Spark Context
sc = spark.sparkContext
sc

<SparkContext master=spark://spark-60245d4a0ac6130d45aab9ea-master-svc.domino-vip2-compute.svc.cluster.local:7077 appName=MyAppName>

In [ ]:
#Profile Data
red_wine.printSchema()
print("Rows: %s" % red_wine.count())

In [5]:
#Feature Extraction into Vectors
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# select the columns to be used as the features (all except `quality`)
featureColumns = [c for c in red_wine.columns if c != 'quality']

# create and configure the assembler
assembler = VectorAssembler(inputCols=featureColumns, 
                            outputCol="features")

# transform the original data
dataDF = assembler.transform(red_wine)
dataDF.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)
 |-- features: vector (nullable = true)



In [6]:
dataDF.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|            features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|[7.4,0.7,0.0,1.9,...|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|[7.8,0.88,0.0,2.6...|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|    

In [7]:
from pyspark.ml.regression import LinearRegression

# fit a `LinearRegression` model using features in colum `features` and label in column `quality`
lr = LinearRegression(maxIter=30, regParam=0.3, elasticNetParam=0.3, featuresCol="features", labelCol="quality")
lrModel = lr.fit(dataDF)

In [8]:
for t in zip(featureColumns, lrModel.coefficients):
    print(t)

('fixed acidity', 0.0)
('volatile acidity', -0.7569245708156528)
('citric acid', 0.0)
('residual sugar', 0.0)
('chlorides', 0.0)
('free sulfur dioxide', 0.0)
('total sulfur dioxide', -0.00020326904604336523)
('density', 0.0)
('pH', 0.0)
('sulphates', 0.2738470451309209)
('alcohol', 0.19463722248782067)


In [9]:
# predict the quality, the predicted quality will be saved in `prediction` column
predictionsDF = lrModel.transform(dataDF)
display(predictionsDF.show())

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|            features|        prediction|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+------------------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|[7.4,0.7,0.0,1.9,...|5.2822415832023255|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|[7.8,0.88,0.0,2.6...| 5.250003816346916|
|          7.8|            0.76|       0.04|      

None

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator

# create a regression evaluator with RMSE metrics

evaluator = RegressionEvaluator(
    labelCol='quality', predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictionsDF)
print("Root Mean Squared Error (RMSE) = %g" % rmse)

Root Mean Squared Error (RMSE) = 0.683189


In [11]:
# split the input data into traning and test dataframes with 70% to 30% weights
(trainingDF, testDF) = red_wine.randomSplit([0.7, 0.3])

In [12]:
from pyspark.ml import Pipeline

# construct the `Pipeline` that with two stages: the `vector assembler` and `regresion model estimator`
pipeline = Pipeline(stages=[assembler, lr])

# train the pipleline on the traning data
lrPipelineModel = pipeline.fit(trainingDF)

# make predictions
traningPredictionsDF = lrPipelineModel.transform(trainingDF)
testPredictionsDF = lrPipelineModel.transform(testDF)

# evaluate the model on test and traning data
print("RMSE on training data = %g" % evaluator.evaluate(traningPredictionsDF))
print("RMSE on test data = %g" % evaluator.evaluate(testPredictionsDF))

RMSE on training data = 0.678189
RMSE on test data = 0.701203


In [13]:
#Run Random Forest
from pyspark.ml.regression import RandomForestRegressor

# define the random forest estimator
rf = RandomForestRegressor(featuresCol="features", labelCol="quality", numTrees=100, maxBins=128, maxDepth=20, \
                           minInstancesPerNode=5, seed=33)
rfPipeline = Pipeline(stages=[assembler, rf])

# train the random forest model
rfPipelineModel = rfPipeline.fit(trainingDF)

In [14]:
#Test Accuracy of Random Forest
rfTrainingPredictions = rfPipelineModel.transform(trainingDF)
rfTestPredictions = rfPipelineModel.transform(testDF)
print("Random Forest RMSE on training data = %g" % evaluator.evaluate(rfTrainingPredictions))
print("Random Forest RMSE on test data = %g" % evaluator.evaluate(rfTestPredictions))

Random Forest RMSE on training data = 0.381986
Random Forest RMSE on test data = 0.601508
